In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from matplotlib.dates import MonthLocator, DateFormatter
from scipy.stats import spearmanr, mannwhitneyu
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.neighbors import KNeighborsRegressor
from sklearn.tree import DecisionTreeRegressor
from sklearn.metrics import mean_squared_error, r2_score
import xgboost as xgb
from sklearn.svm import SVR
from scipy.stats import randint, uniform
from sklearn.model_selection import RandomizedSearchCV

Используйте это, если запускаете не в main.ipynb

In [2]:
# PATH = '../analysing_environmental_issues.csv'
# df = pd.read_csv(PATH, sep=",")
# %run remove_outliers/remove_outliers_run.ipynb

# df = hybrid_outlier_handling(df, df.columns[1:-1])
# %run nans_and_columns/initial_data_handlers_run.ipynb

# convert_to_DateTime(df)

# df = nan_to_interpolate(df)
# df = convert_to_category(df)

In [ ]:
df.describe()

Здесь я копирую из оригинального датафрейма столбцы, вырезанные в ходе предобработки.

In [ ]:
original_df = pd.read_csv("data_imputed_stage4gas_unfilled.csv")

# копируем столбец с опасным газом из оригинального датафрейма
danger_gas = original_df['stage_4_output_danger_gas'].copy()
datetime_column = pd.to_datetime(original_df['DateTime'])

# добавляем столбец в текущий датафрейм
df['stage_4_output_danger_gas'] = danger_gas
df['DateTime'] = datetime_column

In [ ]:
df.describe()

In [ ]:
df.isna()

# Проведение расчётов

## Создать категорию опасности вредного газа от 1 до 3. 
Низкая опасность (1) показатель до 0.05, средняя(2) - от 0.05 до 0.16, высокая(3) - от 0.16

In [7]:
def get_danger_category(value):
    if pd.isna(value):
        return np.nan
    elif value <= 0.05:
        return 1
    elif value <= 0.16:
        return 2
    else:
        return 3

df['danger_category'] = df['stage_4_output_danger_gas'].apply(get_danger_category)

# названия категорий
category_names = {1: 'Низкая', 2: 'Средняя', 3: 'Высокая'}

In [ ]:
# график распределения категорий
plt.figure(figsize=(10, 6))
df['danger_category'].value_counts().sort_index().plot(kind='bar')
plt.title('Распределение категорий опасности газа')
plt.xlabel('Категория опасности')
plt.ylabel('Количество измерений')
plt.xticks(range(3), ['Низкая', 'Средняя', 'Высокая'])
plt.xticks(rotation=0)
plt.show()

## Создать сводную таблицу изменения параметров сгруппированную по дням без учета номера смены. Выведите суммарные показатели входных параметров. При однотипности парамтра проведите суммирования. Сделайте выводы по расходу ресурсов каждый день. Есть ли периоды, когда компания тратила больше ресурсов?

In [9]:
input_params = [col for col in df.columns if 'input' in col]

# группируем по дням и суммируем входные параметры
daily_summary = df.groupby(df['DateTime'].dt.date)[input_params].sum()


In [ ]:
daily_summary.head()

In [ ]:
# # создаем график для каждого входного параметра
# plt.figure(figsize=(15, 10))
# for param in input_params:
#     plt.plot(daily_summary.index, daily_summary[param], label=param, alpha=0.5)
# plt.title('Расход отдельных ресурсов по дням')
# plt.xlabel('Дата')
# plt.ylabel('Расход')
# plt.legend(bbox_to_anchor=(1.05, 1), loc='upper left')

# plt.gca().xaxis.set_major_locator(MonthLocator())
# plt.gca().xaxis.set_major_formatter(DateFormatter('%Y-%m'))
# plt.xticks(rotation=45)

# plt.tight_layout()
# plt.show()

# группируем данные по неделям resample и интерполируем пропущенные значения для читаемости
daily_summary.index = pd.to_datetime(daily_summary.index)
weekly_summary = daily_summary[input_params].resample('W').mean().interpolate(method='linear')

# cоздаем график для каждого входного параметра
plt.figure(figsize=(15, 10))

for param in input_params:
    plt.plot(weekly_summary.index, 
             weekly_summary[param], 
             label=param, 
             alpha=0.7,
             linewidth=2)

plt.title('Расход отдельных ресурсов (еженедельно)')
plt.xlabel('Дата')
plt.ylabel('Средний недельный расход')
plt.legend(bbox_to_anchor=(1.05, 1), loc='upper left', 
          borderaxespad=0.)
# ось X
plt.gca().xaxis.set_major_locator(MonthLocator())
plt.gca().xaxis.set_major_formatter(DateFormatter('%Y-%m'))
plt.xticks(rotation=45)
plt.show()

In [ ]:
# группируем входные параметры (вода и пар) по типу ресурса
water_params = [col for col in input_params if 'water' in col]
steam_params = [col for col in input_params if 'steam' in col]
other_params = [col for col in input_params if not any(x in col for x in ['water', 'steam'])]

# создаем сводную таблицу по типам ресурсов
daily_resource_types = pd.DataFrame(index=daily_summary.index)

# суммируем однотипные ресурсы
daily_resource_types['total_water'] = daily_summary[water_params].sum(axis=1)
daily_resource_types['total_steam'] = daily_summary[steam_params].sum(axis=1)
daily_resource_types['pressure'] = daily_summary['stage_3_input_pressure']
daily_resource_types['polymer'] = daily_summary['stage_4_input_polymer']

####################################################################
# МЕНЕЕ ЧИТАЕМЫЙ, НО БОЛЕЕ ДЕТАЛЬНЫЙ ГРАФИК (СТАРАЯ ВЕРСИЯ)

# # График общего расхода по типам ресурсов
# plt.figure(figsize=(15, 8))
# plt.plot(daily_resource_types.index, daily_resource_types['total_water'], label='Вода', alpha=0.7)
# plt.plot(daily_resource_types.index, daily_resource_types['total_steam'], label='Пар', alpha=0.7)
# plt.gca().xaxis.set_major_locator(MonthLocator())
# plt.gca().xaxis.set_major_formatter(DateFormatter('%Y-%m'))

# # особой информативности как по мне не несёт, но можно оставить
# # plt.plot(daily_resource_types.index, daily_resource_types['pressure'], label='Давление', alpha=0.7) 
# # plt.plot(daily_resource_types.index, daily_resource_types['polymer'], label='Полимеры', alpha=0.7)

# plt.title('Расход основных ресурсов по дням')
# plt.xlabel('Дата')
# plt.ylabel('Объем расхода')
# plt.legend()
# plt.grid(True, alpha=0.3)
# plt.xticks(rotation=45)
# plt.tight_layout()
# plt.show()
####################################################################

# группируем по неделям и интерполируем пропущенные значения (для читаемости)
weekly_resource_types = daily_resource_types.resample('W').mean().interpolate(method='linear')

# График общего расхода по типам ресурсов
plt.figure(figsize=(15, 8))

plt.plot(weekly_resource_types.index, 
         weekly_resource_types['total_water'], 
         label='Вода', 
         alpha=0.7,
         linewidth=2)

plt.plot(weekly_resource_types.index, 
         weekly_resource_types['total_steam'], 
         label='Пар', 
         alpha=0.7,
         linewidth=2)

plt.title('Расход основных ресурсов (еженедельно)')
plt.xlabel('Дата')
plt.ylabel('Средний недельный расход')
plt.legend()
plt.grid(True, alpha=0.3)

# Настройка оси X
plt.gca().xaxis.set_major_locator(MonthLocator())
plt.gca().xaxis.set_major_formatter(DateFormatter('%Y-%m'))
plt.xticks(rotation=45)

plt.tight_layout()
plt.show()

### Мне показалось, что не будет лишним также создать отдельные группы под потребление на каждом этапе производства

In [ ]:

# группировка параметров по стадиям
stage_2_params = [col for col in input_params if 'stage_2' in col]
stage_3_params = [col for col in input_params if 'stage_3' in col]
stage_4_params = [col for col in input_params if 'stage_4' in col]

# создаем сводные таблицы по стадиям
daily_stages = pd.DataFrame(index=daily_summary.index)
daily_stages['stage_2'] = daily_summary[stage_2_params].sum(axis=1)
daily_stages['stage_3'] = daily_summary[stage_3_params].sum(axis=1)
daily_stages['stage_4'] = daily_summary[stage_4_params].sum(axis=1)

####################################################################
# МЕНЕЕ ЧИТАЕМЫЕ, НО БОЛЕЕ ДЕТАЛЬНЫЕ ГРАФИКИ (СТАРАЯ ВЕРСИЯ)

# #  график по каждой стадии, суммарное потребление НЕ разделённое на категории (вода/пар/полимеры)
# plt.figure(figsize=(15, 8))
# for stage in daily_stages.columns:
#     plt.plot(daily_stages.index, daily_stages[stage], label=f'Стадия {stage[-1]}', alpha=0.7)
# plt.title('Потребление ресурсов по стадиям производства')
# plt.xlabel('Дата')
# plt.ylabel('Суммарный расход')
# plt.legend()
# plt.grid(True, alpha=0.3)
# # цена деления - месяц
# plt.gca().xaxis.set_major_locator(MonthLocator())
# plt.gca().xaxis.set_major_formatter(DateFormatter('%Y-%m'))
# plt.xticks(rotation=45)
# plt.tight_layout()
# plt.show()


# fig, axes = plt.subplots(3, 1, figsize=(15, 15))
# #  график по каждой стадии,  потребление по категориям (вода/пар/полимеры)
# for idx, (stage_name, params) in enumerate(zip(['stage_2', 'stage_3', 'stage_4'], [stage_2_params, stage_3_params, stage_4_params])):
#     for param in params:
#         axes[idx].plot(daily_summary.index, daily_summary[param], label=param, alpha=0.7)
#     axes[idx].set_title(f'Потребление ресурсов на стадии {stage_name[-1]}')
#     axes[idx].set_xlabel('Дата')
#     axes[idx].set_ylabel('Расход')
#     axes[idx].legend()
#     axes[idx].grid(True, alpha=0.3)
#     # устанавливаем цену деления
#     axes[idx].xaxis.set_major_locator(MonthLocator())
#     axes[idx].xaxis.set_major_formatter(DateFormatter('%Y-%m'))
#     axes[idx].tick_params(axis='x', rotation=45)

# plt.tight_layout()
# plt.show()

####################################################################

# стата по стадиям
print("\nстатистика расхода по стадиям:")
print(daily_stages.describe())


In [ ]:
print('=' * 50)
print('ГРАФИКИ ЕЖЕНЕДЕЛЬНОГО ПОТРЕБЛЕНИЯ')
weekly_stages = daily_stages.resample('W').mean()

# для графика интерполируем отсутствующие значения
weekly_stages_interpolated = weekly_stages.interpolate(method='linear')

# график по стадиям resample по неделям и интерполяцией
plt.figure(figsize=(15, 8))
for stage in weekly_stages_interpolated.columns:
    plt.plot(weekly_stages_interpolated.index, weekly_stages_interpolated[stage], 
             label=f'Стадия {stage[-1]}', alpha=0.7, linewidth=2)
    
plt.title('Потребление ресурсов по стадиям производства (еженедельно)')
plt.xlabel('Дата')
plt.ylabel('Средний недельный расход')
plt.legend()
plt.grid(True, alpha=0.3)
plt.gca().xaxis.set_major_locator(MonthLocator())
plt.gca().xaxis.set_major_formatter(DateFormatter('%Y-%m'))
plt.xticks(rotation=45)
plt.tight_layout()
plt.show()

# графики по категориям с интерполяцией
fig, axes = plt.subplots(3, 1, figsize=(15, 15))

for idx, (stage_name, params) in enumerate(zip(['stage_2', 'stage_3', 'stage_4'], 
                                             [stage_2_params, stage_3_params, stage_4_params])):
    weekly_data = daily_summary[params].resample('W').mean()
    # Интерполируем пропущенные значения для каждого параметра
    weekly_data_interpolated = weekly_data.interpolate(method='linear')
    
    for param in params:
        axes[idx].plot(weekly_data_interpolated.index, weekly_data_interpolated[param], 
                      label=param, alpha=0.7, linewidth=2)
    
    axes[idx].set_title(f'Потребление ресурсов на стадии {stage_name[-1]} (еженедельно)')
    axes[idx].set_xlabel('Дата')
    axes[idx].set_ylabel('Средний недельный расход')
    axes[idx].legend()
    axes[idx].grid(True, alpha=0.3)
    axes[idx].xaxis.set_major_locator(MonthLocator())
    axes[idx].xaxis.set_major_formatter(DateFormatter('%Y-%m'))
    axes[idx].tick_params(axis='x', rotation=45)

plt.tight_layout()
plt.show()

Находим дни с максимальным суммарным потреблением. Пока что видно, что наиболее ресурсозатратным периодом является первая половина 2021. (Однако нельзя игнорировать факт, что она также является наиболее задокументированной).

In [ ]:
# дни с максимальным потреблением по стадиям
print("\n 10 дней с максимальным потреблением")
for stage in daily_stages.columns:
    peak_days = daily_stages[stage].nlargest(10)
    print(f"\n{stage}:")
    print(peak_days)

In [ ]:
daily_stages.head(20)

Попробуем проанализировать периоды повышенного потребления ресурсов. Для этого я считаю "средний расход" за всё время (увы, работаем с тем, что есть...)
Смотрю, насколько сильно реальный расход отличается от среднего и определяю порог высокого и критично выского потребления, используя Z-score (мера относительного разброса наблюдаемого или измеренного значения, которая показывает сколько стандартных отклонений составляет его разброс относительного среднего значения).
Если расход сильно выше среднего несколько дней подряд - это явно проблема и, т.н "тренд", следовательно - стоит уделить этому периоду особое внимание.

In [ ]:
def analyze_daily_resource_usage(daily_summary):

    daily_summary.index = pd.to_datetime(daily_summary.index)
    
    # рассчитываем общее потребление ресурсов за день
    daily_total = daily_summary.sum(axis=1)
    
    mean_usage = daily_total.mean()
    std_usage = daily_total.std()
    
    # определяем пороги для категорий потребления (используем z-score)
    high_consuming = mean_usage + std_usage
    very_high_consuming = mean_usage + 2*std_usage
    
    # категоризируем дни по уровню потребления
    use_category = pd.Series(index=daily_total.index, dtype='object')
    use_category[daily_total <= mean_usage] = 'Normal'
    use_category[(daily_total > mean_usage) & (daily_total <= high_consuming)] = 'Above Average'
    use_category[(daily_total > high_consuming) & (daily_total <= very_high_consuming)] = 'High'
    use_category[daily_total > very_high_consuming] = 'Very High'
    
    # найдем периоды повышенного потребления
    high_usage_periods = []
    current_period = []
    
    for date, category in use_category.items():
        if category in ['High', 'Very High']:
            current_period.append(date)
        elif len(current_period) > 0:
            if len(current_period) >= 2:  # период 2 дня
                high_usage_periods.append({
                    'start': min(current_period),
                    'end': max(current_period),
                    'duration': len(current_period),
                    'average_usage': daily_total[current_period].mean()
                })
            current_period = []
            
    # добавим последний период, если он существует
    if len(current_period) >= 2:
        high_usage_periods.append({
            'start': min(current_period),
            'end': max(current_period),
            'duration': len(current_period),
            'average_usage': daily_total[current_period].mean()
        })
    
    # метод ресэмпл (определение из гугла):
    # О методе .resample() можно думать как о группировке по времени, за которым следует метод агрегирования для каждой из его групп.
    # Другими словами - это GroupBy основанный на времени. Его можно использовать непосредственно из объектов обычной группировки DataFrameGroupBy.

    monthly_avg = daily_total.resample('M').mean()
    monthly_trend = monthly_avg.pct_change()
    
    
    
    return {'daily_stats': {
            'mean': mean_usage,
            'std': std_usage,
            'min': daily_total.min(),
            'max': daily_total.max(),
            'median': daily_total.median()
        }, 'use_category': use_category, 'high_usage_periods': high_usage_periods, 'monthly_trend': monthly_trend, 'daily_total': daily_total
    }


def print_analysis_results(results): #красивый вывод результата
    print("\n___ Статистика ежедневного потребления ресурсов: ___\n")
    print('------------------------------------------------------------')
    print(f"Среднее потребление: {results['daily_stats']['mean']:.2f}")
    print(f"Стандартное отклонение: {results['daily_stats']['std']:.2f}")
    print(f"Минимальное потребление: {results['daily_stats']['min']:.2f}")
    print(f"Максимальное потребление: {results['daily_stats']['max']:.2f}")
    print(f"Медианное потребление: {results['daily_stats']['median']:.2f}")
    print('------------------------------------------------------------')  

    print("\n___ Периоды повышенного потребления: ___")
    for period in results['high_usage_periods']:
        print(f"\nПериод: {period['start'].date()} - {period['end'].date()}")
        print(f"Длительность: {period['duration']} дней")
        print(f"Среднее потребление: {period['average_usage']:.2f}")
    print('------------------------------------------------------------')    



results = analyze_daily_resource_usage(daily_summary)
print_analysis_results(results)

**В итоге, следующие периоды имели тренд на повышенное потребление суммарных ресурсов:**

1) 2021-11-22 - 2021-12-03 (12 дней)
2) 2021-12-21 - 2021-12-28 (8 дней)
3) 2022-02-22 - 2022-03-01 (8 дней)

Немного ранее мы заметили, что сильные скачки потребления приходились на первую половину 2021 (опираясь на максимальные значения потребления). Они тоже есть в списке периодов повышенного потребления! Однако их длительность не превышает двух дней, т.е есть смысл предполагать, что имели место быть ВСПЫШКИ повышенного потребления ресурсов.

## Создать сводную таблицу изменения категории опасности сгруппированную по месяцам. Выводите ту категорию, которая была чаще всего в месяца. Есть ли в течении года периоды, когда производство было самым опасным?


In [17]:
# создаем сводную таблицу изменения категории опасности (группировка по месяцам)
monthly_summary = df.groupby(df['DateTime'].dt.strftime('%Y-%m')).agg({'danger_category': lambda x: x.mode().iloc[0] if not x.mode().empty else np.nan, 'stage_4_output_danger_gas': 'mean', 'DateTime': 'count'
}).rename(columns={'danger_category': 'most_common_category',
    'stage_4_output_danger_gas': 'avg_danger_gas',
    'DateTime': 'count' # важный столбец!
})


monthly_summary['most_common_category'] = monthly_summary['most_common_category'].map(category_names)



In [ ]:
print("Сводная таблица по месяцам:")
monthly_summary


**После группировки данных по месяцу, видны некоторые особенности распределения количества наблюдений, в частности:**

1. полностью отсутствуют данные за некоторые месяцы
2. крупный разброс в количестве измерений по месяцам (варьируется от 20 до 418.)

**Что с этим делать?**
Есть вариант добавить "веса" для тех месяцев, что более детально отображают реальное положение дел на производстве (наблюдений больше), либо как-нибудь поделить временные промежутки, чтобы "уровнять" данные... Не знаю пока, в общем

In [ ]:
# также можем добавить проверку распределения категорий опасности по каждому месяцу
print("\nРаспределение категорий по месяцам:")
monthly_distribution = df.groupby(df['DateTime'].dt.strftime('%Y-%m'))['danger_category'].value_counts()
for month_year, category_counts in monthly_distribution.groupby(level=0):
    print(month_year)
    print(category_counts)
    print("----------------------------")

**"Рекордсмены" по абсолютному (!!! т.е без учета численности выборки и тд) количеству наблюдений с отметкой о высокой опасности газа:**

1) 2021-02 (22)
2) 2021-12 (40)
3) 2022-03 (27)

## Для каждой категории вывести среднее значение каждого параметра (без учета номера смены). Есть ли заметные отличия типа категории?

**Нам необходимо вывести среднее значение каждого параметра без учета смены. Для наглядности сделаем табличку средних значений (и табличку динамики в процентах, т.е как меняется ср. значение с категорией) и построим графики**

In [ ]:
def analyze_data(df, category_names):
    numeric_cols = df.select_dtypes(include=[np.number]).columns
    parameters = [col for col in numeric_cols if col not in ['work_shift', 'stage_4_output_danger_gas', 'danger_category']]
    
    # создаем базовую таблицу средних значений
    means = df.groupby('danger_category')[parameters].mean().round(2)
    means.index = means.index.map(category_names)
    
    # изменения процентные
    changes = pd.DataFrame(index=parameters)
    changes['Low_to_Med_%'] = ((means.loc['Средняя'] / means.loc['Низкая'] - 1) * 100).round(1)
    changes['Med_to_High_%'] = ((means.loc['Высокая'] / means.loc['Средняя'] - 1) * 100).round(1)
    
    # не суди да не судим будешь...
    param_description = {
        'stage_1_output_konv_avd': 'Стадия 1 / Конверсия сырья',
        'stage_2_input_water_sum': 'Стадия 2 / Подача воды',
        'stage_2_output_bottom_pressure': 'Стадия 2 / Давление низа',
        'stage_2_output_bottom_temp': 'Стадия 2 / Температура низа',
        'stage_2_output_bottom_temp_hum_steam': 'Стадия 2 / Температура пара низа',
        'stage_2_output_bottom_vacuum': 'Стадия 2 / Вакуум низа',
        'stage_2_output_top_pressure': 'Стадия 2 / Давление верха',
        'stage_2_output_top_pressure_at_end': 'Стадия 2 / Давление верха на выходе',
        'stage_2_output_top_temp': 'Стадия 2 / Температура верха',
        'stage_2_output_top_vacuum': 'Стадия 2 / Вакуум верха',
        'stage_3_input_pressure': 'Стадия 3 / Подаваемое давление',
        'stage_3_input_soft_water': 'Стадия 3 / Подача смягченной воды',
        'stage_3_input_steam': 'Стадия 3 / Подача пара',
        'stage_3_output_temp_hum_steam': 'Стадия 3 / Температура пара',
        'stage_3_output_temp_top': 'Стадия 3 / Температура верха',
        'stage_4_input_overheated_steam': 'Стадия 4 / Подача перегретого пара',
        'stage_4_input_polymer': 'Стадия 4 / Подача полимера',
        'stage_4_input_steam': 'Стадия 4 / Подача пара',
        'stage_4_input_water': 'Стадия 4 / Подача воды',
        'stage_4_output_dry_residue_avg': 'Стадия 4 / Сухой остаток',
        'stage_4_output_product': 'Стадия 4 / Готовый продукт'
    }
    
    # переименовываем параметры
    result = means.copy()
    result.columns = [param_description[col] for col in parameters]
    result = result.T
    
    changes.index = [param_description[col] for col in parameters]
    
    # пробегаем по стадиям и делаем анализ категорий
    for stage in range(1, 5):
        stage_params = [p for p in result.index if f'Стадия {stage}' in p]
        if stage_params:
            print("===========================================")
            print(f"СТАДИЯ {stage}")
            print("===========================================")
            
            # график
            plt.figure(figsize=(12, 6))
            x = np.arange(len(stage_params))
            width = 0.25
            
            plt.bar(x - width, result.loc[stage_params, 'Низкая'], width, label='Низкая', alpha=0.8)
            plt.bar(x, result.loc[stage_params, 'Средняя'], width, label='Средняя', alpha=0.8)
            plt.bar(x + width, result.loc[stage_params, 'Высокая'], width, label='Высокая', alpha=0.8)
            
            plt.title(f'Стадия {stage}: Параметры по категориям')
            plt.xticks(x, [p.split('/')[1].strip() for p in stage_params], rotation=45, ha='right')
            plt.legend()
            plt.grid(True, alpha=0.3)
            plt.tight_layout()
            plt.show()
            
            # анализ каждого параметра
            for param in stage_params:
                low_val = result.loc[param, 'Низкая']
                med_val = result.loc[param, 'Средняя']
                high_val = result.loc[param, 'Высокая']
                
                low_to_med = ((med_val/low_val) -1)* 100
                med_to_high = ((high_val/med_val)-1)*100
                
                print(f"\n{param.split('/')[1].strip()}:")
                print(f"Низкая категория: {low_val:.2f}")
                print(f"Средняя категория: {med_val:.2f} (изменение: {low_to_med:.1f}%)")
                print(f"Высокая категория: {high_val:.2f} (изменение: {med_to_high:.1f}%)")
                
                if abs(low_to_med) > 5 or abs(med_to_high) > 5:
                    print("!!! Значимое изменение !!!")
    
    return result, changes


result, changes = analyze_data(df, category_names)

**Таблицы:**

In [ ]:
changes

In [ ]:
result

**Анализируя данные, можно выделить основные моменты:**

**1) Наиболее важные изменения происходят на 4-й стадии производства:**

1. Подача воды увеличивается (+28%)! при средней опасности
2. Подача пара возрастает на 18%
3. Подача полимера повышается на 7%
4. Количество готового продукта падает на 4% при высокой опасности производства!!!


**2) На 2 стадии:**

1. Подача воды увеличивается на 11%
2. Давление низа возрастает на 6%
3. Немножко снижается вакуум верха (-4.0% и -4.5%)


**3) На 3 стадии:**

1. Подача смягченной воды снижается на 9.1%


**4) На 1-й стадии изменения самые незначительные:**

1. Конверсия сырья увеличивается на 4.7%

**Промежуточный вывод таков:** наибольшее влияние на опасность газа оказывает повышенная подача воды и пара на 4-й стадии. При этом эффективность производства падает (меньше готового продукта).